In [1]:
# import data
MPST_FILE = "data/mpst/mpst_full_data.csv"
MD_DIR = "data/movie_dataset"
OSCAR_FILE = "data/the_oscar_award.csv"

In [2]:
import pymongo
import json 
import pandas as pd

In [3]:
URI = "mongodb://127.0.0.1:27017/?readPreference=primary&ssl=false"
client = pymongo.MongoClient(URI)

In [29]:
mpst_df = pd.read_csv(MPST_FILE)
mpst_df = mpst_df.rename(columns={"imdb_id":"_id"})#for mongo
mpst_df[mpst_df._id.duplicated()]#no duplicated ids
mpst_df = mpst_df.set_index("_id")
mpst_df.head()

,title,plot_synopsis,tags,split,synopsis_source
_id,,,,,
tt0057603,I tre volti della paura,Note: this synopsis is for the orginal Italian...,"cult, horror, gothic, murder, atmospheric",train,imdb
tt1733125,Dungeons & Dragons: The Book of Vile Darkness,"Two thousand years ago, Nhagruul the Foul, a s...",violence,train,imdb
tt0033045,The Shop Around the Corner,"Matuschek's, a gift store in Budapest, is the ...",romantic,test,imdb
tt0113862,Mr. Holland's Opus,"Glenn Holland, not a morning person by anyone'...","inspiring, romantic, stupid, feel-good",train,imdb
tt0086250,Scarface,"In May 1980, a Cuban man named Tony Montana (A...","cruelty, murder, dramatic, cult, violence, atm...",val,imdb


In [30]:
#separate tags
tags_df = mpst_df[["tags"]].reset_index()
#separate metadata
meta_df = mpst_df[["split","synopsis_source"]].reset_index()
mpst_df = mpst_df.drop(columns=["split","synopsis_source","tags"]).reset_index()

In [31]:
#create db for the data set
db = client['MPST']
db['plots'].insert_many(mpst_df.to_dict("records"))
db['tags'].insert_many(tags_df.to_dict("records"))
db['metadata'].insert_many(meta_df.to_dict("records"))

In [32]:
## MOvie dataset

In [4]:
import os 
data = {}
for f in os.listdir(MD_DIR):
    name = f.split(".")[0]
    if "small" not in name: #load full dataset
        path = os.path.join(MD_DIR,f)
        data[name] = pd.read_csv(path)

In [5]:
data.keys()

dict_keys(['credits', 'keywords', 'links', 'movies_metadata', 'ratings'])

In [6]:
data['movies_metadata'] = data['movies_metadata'].drop(columns=["homepage","imdb_id"])
#remove duplicated id
data['movies_metadata'] = data['movies_metadata'][~data['movies_metadata'].id.duplicated()]
#some id are dates (other fields are wrong too, just remove )
data['movies_metadata'] = data['movies_metadata'][~data['movies_metadata'].id.str.contains('-')]
data['movies_metadata'].id = data['movies_metadata'].id.astype(int) 
data['movies_metadata'].head()
#make a separate table for genres and keep only id 
#the same for spoken_languanges and belogns_to_collection

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,poster_path,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [212]:
genres = {}
genres_ids = []
for l in data['movies_metadata'].genres.values:
    l = eval(l)
    tmp = []
    for g in l:
        tmp.append(g['id'])
        #TODO: check id univoco
        if g['id'] not in genres:
            genres[g['id']] = g['name']
    genres_ids.append(tmp)
data['movies_metadata'].genres = genres_ids
genres_df = pd.DataFrame(genres.items(), columns=['_id','name'])
genres_df.head()

,_id,name
0,16,Animation
1,35,Comedy
2,10751,Family
3,12,Adventure
4,14,Fantasy


In [213]:
langs = {}
langs_ids = []
for l in data['movies_metadata'].spoken_languages.values:
    if type(l) == str:
        l = eval(l)
        tmp = []
        for g in l:
            tmp.append(g['iso_639_1'])
            if g['iso_639_1'] not in genres:
                langs[g['iso_639_1']] = g['name']
    else:
        tmp = float("nan")
    langs_ids.append(tmp)
data['movies_metadata'].spoken_languages = langs_ids
#langs_df = pd.DataFrame(langs.items(), columns=['_id','name'])
#langs_df.head()

In [214]:
collections = {}
collections_ids = []
for l in data['movies_metadata'].belongs_to_collection.values:
    tmp = None
    #print(type(l))
    if type(l) == str:
        l = eval(l)
        #print(l)
        if type(l)==dict:
            tmp = int(l['id'])
            if tmp not in collections:
                collections[tmp] = l['name']

    collections_ids.append(tmp)
data['movies_metadata'].belongs_to_collection = collections_ids
collections_df = pd.DataFrame(collections.items(), columns=['_id','name'])
collections_df.head()

,_id,name
0,10194,Toy Story Collection
1,119050,Grumpy Old Men Collection
2,96871,Father of the Bride Collection
3,645,James Bond Collection
4,117693,Balto Collection


In [215]:
#data['movies_metadata'] = data['movies_metadata'].rename(columns={'id','_id'})
#data['movies_metadata'].id = data['movies_metadata'].id.astype(int)
#data['movies_metadata'].adult = data['movies_metadata'].adult.apply(lambda x: x.lower().strip() == "true")
from collections.abc import Iterable 
def f(s):
    if type(s)!=str:
        return None
    l = eval(s)
    if not isinstance(l, Iterable):
        return None
    return [c['iso_3166_1'] for c in l]
data['movies_metadata'].production_countries = data['movies_metadata'].production_countries.apply(f)
data['movies_metadata'].head()

,adult,belongs_to_collection,budget,genres,id,original_language,original_title,overview,popularity,poster_path,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,10194.0,30000000,"[16, 35, 10751]",862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,1995-10-30,373554033.0,81.0,[en],Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[12, 14, 10751]",8844,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,...,1995-12-15,262797249.0,104.0,"[en, fr]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,119050.0,0,"[10749, 35]",15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,...,1995-12-22,0.0,101.0,[en],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[35, 18, 10749]",31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,...,1995-12-22,81452156.0,127.0,[en],Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,96871.0,0,[35],11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,...,1995-02-10,76578911.0,106.0,[en],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [216]:
#data['movies_metadata'].budget = pd.to_numeric(data['movies_metadata'].budget, errors='coerce' )
#data['movies_metadata'].release_date = pd.to_datetime(data['movies_metadata'].release_date, errors='coerce')
#data['movies_metadata'].release_date = data['movies_metadata'].release_date.dt.date
#data['movies_metadata'].release_date[data['movies_metadata'].release_date.isnull()] = None
#data['movies_metadata']
#TODO: fix NaT problems with mongo

In [217]:
comp = {}
comp_ids = []
for v in data['movies_metadata'].production_companies:
    ids = None
    if type(v) == str:
        prod_comps = eval(v)
        if prod_comps:
            #print(prod_comps)
            ids = []
            for pc in prod_comps:
                if pc['id'] not in comp:
                    comp[pc['id']] = pc['name']
                ids.append(pc['id'])
    comp_ids.append(ids)
data['movies_metadata'].production_companies = comp_ids
prod_comp_df = pd.DataFrame(comp.items(), columns=['_id','name'])
prod_comp_df.head()

,_id,name
0,3,Pixar Animation Studios
1,559,TriStar Pictures
2,2550,Teitler Film
3,10201,Interscope Communications
4,6194,Warner Bros.


In [218]:
data['ratings'].head()
#no unique index. TODO:cast timestamp to datetime
#pd.to_datetime(data['ratings'].timestamp

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [219]:
data['links'].head()
#the dataset id, internet movies db id, the movie db id
#for join other dataset

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [220]:
keywords = {}
movies = {}
for i,r in data['keywords'].iterrows():
    l = []
    mv_id = r['id']
    for kw in eval(r['keywords']):
        if kw['id'] not in keywords:
            keywords[kw['id']] = kw['name'], [mv_id]
        else:
            keywords[kw['id']][1].append(mv_id) 
    
#TODO: join with metadata?
kw_df = pd.DataFrame(((a,*b) for a,b in keywords.items()), columns=['_id','name','movie_ids'])
kw_df.head()

,_id,name,movie_ids
0,931,jealousy,"[862, 16420, 35196, 649, 18183, 2307, 888, 275..."
1,4290,toy,"[862, 11934, 9279, 9714, 10585, 11597, 1857, 9..."
2,5202,boy,"[862, 73067, 10895, 50850, 8838, 24808, 5332, ..."
3,6054,friendship,"[862, 902, 400, 8839, 9382, 236, 10464, 888, 1..."
4,9713,friends,"[862, 16420, 10474, 8839, 9382, 1024, 48287, 9..."


In [221]:
#data['credits'].iloc[0,1]
#cast: actor/character
#crew: director, producer, interns
# -> new collection -> People{_id :int, name :str,gender :int, jobs :List[str]}
# nb if person is in cast one job is actor
# Credits{
#   cast: List[{
#               cast_id :int, character :str, credit_id :str, 
#               order :int, person_id :People._id
#         }],
#   crew: List[{
#               cast_id :int, departement :str, credit_id :str, 
#               job :str, person_id :People._id
#         }],
# }

In [222]:
people = {}
cast = []
for l in data['credits'].cast.values:
    tmp = None
    #print(type(l))
    if type(l) == str:
        l = eval(l)
        for p in l:
            tmp = int(p['id'])
            if tmp not in collections:
                people[tmp] = p['name'], p['gender'], p['profile_path'], ['Actor']
            
            p['person_id'] = p['id']
            del p['name'], p['gender'], p['profile_path'], p['id']
        cast.append(l)
    else:
        cast.append(v)

crew = []
for v in data['credits'].crew.values:
    tmp = None
    #print(type(l))
    if type(v) == str:
        l = eval(v)
        for p in l:
            tmp = int(p['id'])
            if tmp not in people:
                people[tmp] = p['name'], p['gender'], p['profile_path'], [p['job']]
            else:
                if p['job'] not in people[tmp][3]:
                    people[tmp][3].append(p['job'])
            
            p['person_id'] = p['id']
            del p['name'], p['gender'], p['profile_path'], p['id'], p['job']
        crew.append(l)
    else:
        crew.append(v)
    
#TODO: remove people info and leave only id inside metadata
people_df = pd.DataFrame( ((a,*b) for a,b in people.items()), columns=['_id','name','gender','profile_path','jobs'])
people_df.head()


,_id,name,gender,profile_path,jobs
0,31,Tom Hanks,2,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg,"[Actor, Director, Screenplay, Producer, Execut..."
1,12898,Tim Allen,2,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg,"[Actor, Producer, Director]"
2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg,[Actor]
3,12899,Jim Varney,2,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg,"[Actor, In Memory Of]"
4,12900,Wallace Shawn,2,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg,"[Actor, Writer]"


In [223]:
#create db for the data set
db = client['movie_dataset']

In [224]:
%timeit
crew_df, cast_df = [], []
for i, r in data['credits'].iterrows():
    df = pd.DataFrame(cast[i])
    df['movie_id'] = [r['id']]*len(df)
    cast_df.append(df)

    df = pd.DataFrame(crew[i])
    df['movie_id'] = [r['id']]*len(df)
    crew_df.append(df)
    if i % 200 == 0: # load every 200 movies -> save memory (hopefully)
        crew_df, cast_df = pd.concat(crew_df), pd.concat(cast_df)
        db['cast'].insert_many(cast_df.to_dict("records"))
        db['crew'].insert_many(crew_df.to_dict("records"))
        crew_df, cast_df = [], []

if len(crew_df) > 0: 
    crew_df, cast_df = pd.concat(crew_df), pd.concat(cast_df)
    db['cast'].insert_many(cast_df.to_dict("records"))
    db['crew'].insert_many(crew_df.to_dict("records"))

In [225]:
db['ids'].insert_many(data['links'].rename(columns={'movieId':'_id'}).to_dict("records"))
db['genres'].insert_many(genres_df.to_dict("records"))
db['collections'].insert_many(collections_df.to_dict("records"))
db['metadata'].insert_many(data['movies_metadata'].rename(columns={'id':'_id'}).to_dict("records"))
db['production_companies'].insert_many(prod_comp_df.to_dict("records"))
db['people'].insert_many(people_df.to_dict("records"))
db['keywords'].insert_many(kw_df.to_dict("records"))

#raise Exception("not complete")

#batchs
SIZE = 5000
limit = len(data['ratings'])
print("Loading ratings by batches..")
for i in range(0, limit, SIZE):
    print(f"{'x' if i%2==0 else '+' }", end="\r")
    db['ratings'].insert_many(data['ratings'].iloc[i:i+SIZE].to_dict("records")) #BIG
print()
db['ratings'].insert_many(data['ratings'].iloc[i+SIZE:].to_dict("records"))

Loading ratings by batches..
26015000/5203


In [ ]:
#OSCARS
#TODO: find movie id and person id
oscar_df = pd.read_csv(OSCAR_FILE)
db['oscar_awards'].insert_many(oscar_df.to_dict("records"))

In [229]:
client.close()